In [1]:
import pandas as pd
import argilla as rg

data_file = "data/data_gn.pkl"

dataset = pd.read_pickle(data_file)



In [2]:
def to_conll_eval_format(data):
    output = []
    for d in data:
        sentence_tags = []
        for token in d['tokens']:
            sentence_tags.append(token['tag'])
        output.append(sentence_tags)
    return output

tokens = dataset["tokens"].tolist()
conll = to_conll_eval_format(dataset["metrics"].tolist())



In [3]:
conll_total = [c for s in conll for c in s]
set(conll_total)

{'B-NG', 'I-NG', None, 'O'}

In [4]:
# Remove all sentences in conll_total that 

In [5]:
import pandas as pd
data = pd.DataFrame({"tokens": tokens, "ng_tags": conll})


In [6]:
data

# If any element in ng_tags is None, remove the whole row
data["has_none"] = data["ng_tags"].apply(lambda x: None in x)

# Count has_none
print(data["has_none"].value_counts())

# Drop rows with has_none = True
data = data[data["has_none"] == False].reset_index(drop=True)

False    2606
True        7
Name: has_none, dtype: int64


In [7]:
data = data.reset_index()
# Rename index to id
data = data.rename(columns={"index": "id"})
#data = data.to_dict('records')


In [16]:
del data["has_none"]

In [8]:
# Len of dataset

# data["len_t"] = data["tokens"].apply(lambda x: len(x))
# data["len_ng"] = data["ng_tags"].apply(lambda x: len(x))

# # Check if len_t == len_ng and count the number of sentences that are not equal
# data["len_equal"] = data["len_t"] == data["len_ng"]
# data["len_equal"].value_counts()

In [9]:
#features = Features({"ng_tags": Sequence(ClassLabel(num_classes=3, names=['B-NG', 'I-NG', 'O']))})

#hf_dataset = Dataset.from_list(data, features=features)

In [18]:
from datasets import Dataset, Features, ClassLabel, Sequence, Value

hf_dataset = Dataset.from_pandas(data)

new_features = Features(
    {
        'id': Value(dtype='int64', id=None),
        'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
        'ng_tags': Sequence(feature=ClassLabel(names=['B-NG', 'I-NG', 'O'], id=None), length=-1, id=None),
    }
)

# Cast the dataset with the updated features.
hf_dataset = hf_dataset.cast(new_features)
hf_dataset

Casting the dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

Dataset({
    features: ['id', 'tokens', 'ng_tags'],
    num_rows: 2606
})

In [19]:
hf_dataset.features["ng_tags"].feature.names


['B-NG', 'I-NG', 'O']

In [20]:
hf_dataset["ng_tags"]

[[0,
  0,
  0,
  2,
  2,
  0,
  2,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  1,
  1,
  2,
  2,
  0,
  1,
  1,
  1,
  1,
  2,
  2,
  0,
  2,
  2,
  0,
  2],
 [0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 2, 0, 1, 2, 0, 2, 0],
 [2,
  0,
  0,
  1,
  2,
  2,
  2,
  2,
  2,
  0,
  1,
  2,
  2,
  2,
  0,
  1,
  2,
  2,
  2,
  2,
  2,
  0,
  1,
  2,
  2,
  2,
  0,
  1,
  2,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  0,
  2,
  2,
  2,
  2,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2],
 [0,
  0,
  2,
  0,
  1,
  1,
  2,
  0,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  1,
  2,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  0,
  2,
  0,
  1,
  2,
  0,
  1,
  2,
  2,
  0,
  1,
  2,
  2,
  2,
  0,
  1,
  2,
  2],
 [0,
  2,
  2,
  2,
  0,
  1,
  1,
  2,
  0,
  1,
  1,
  1,
  2,
  0,
  2,
  2,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  0,
  1,
  1,
  2],
 [0,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  0,
  2,
  0,
  2,

In [21]:
hf_dataset = hf_dataset.train_test_split(test_size=0.2, shuffle=True, seed=13)
# Rename datasets["test"] to datasets["validation"]
hf_dataset["validation"] = hf_dataset["test"]
del hf_dataset["test"]
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ng_tags'],
        num_rows: 2084
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ng_tags'],
        num_rows: 522
    })
})

In [22]:
hf_dataset.push_to_hub("spanish_nominal_groups_conll2003")

[06/12/23 22:18:32] WARNING  WARNING:datasets.dataset_dict:Pushing split train to the Hub.     ]8;id=439832;file:///home/jlortiz/data/anaconda3/envs/py310/lib/python3.10/site-packages/datasets/dataset_dict.py\dataset_dict.py]8;;\:]8;id=537983;file:///home/jlortiz/data/anaconda3/envs/py310/lib/python3.10/site-packages/datasets/dataset_dict.py#1532\1532]8;;\

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

[06/12/23 22:18:37] WARNING  WARNING:datasets.dataset_dict:Pushing split validation to the     ]8;id=965701;file:///home/jlortiz/data/anaconda3/envs/py310/lib/python3.10/site-packages/datasets/dataset_dict.py\dataset_dict.py]8;;\:]8;id=974007;file:///home/jlortiz/data/anaconda3/envs/py310/lib/python3.10/site-packages/datasets/dataset_dict.py#1532\1532]8;;\
                             Hub.                                                                                  

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]